In [1]:
import pandas as pd
import numpy as np

In [2]:
def clean_data(data, price_data=None, on_chain_data=None, set_index=True):
    if price_data and on_chain_data:
        raise ValueError('Invalid Parameter Values: Both price_data or on_chain data cannot be True')
    elif price_data:
        prices = data.copy()
        if prices.isna().any().any():
            print('you have nans here')
            return prices 
        
        if set_index:
            prices['time_period_end'] = pd.to_datetime(prices['time_period_end'])
            prices = prices.set_index('time_period_end') 
        else:
            prices.index = pd.to_datetime(prices.index)

        prices['time_open'] = pd.to_datetime(prices['time_open'])
        prices['time_close'] = pd.to_datetime(prices['time_close'])
        return prices
    
    elif on_chain_data:
        metrics = data.copy()
        if metrics.isna().any().any():
            print('you have nans here')
            return metrics 
        
        if set_index:
            metrics['hour'] = pd.to_datetime(metrics['hour'])
            metrics = metrics.set_index('hour')
        else:
            metrics.index = pd.to_datetime(metrics.index)
        return metrics
    else:
        raise ValueError('Invalid Parameter Values: price_data or on_chain data must be True')
    

In [3]:
def check_missing_hours(df):
    """
    Check if the DataFrame's datetime index skips any hourly datapoints.
    
    Parameters:
        df (pd.DataFrame): DataFrame with a DatetimeIndex.
        
    Returns:
        missing (pd.DatetimeIndex): The missing hourly timestamps.
    """
    # Ensure the index is a DatetimeIndex
    if not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError("DataFrame index must be a DatetimeIndex.")
    
    # Create an expected date_range from the minimum to the maximum timestamp at hourly frequency
    expected_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='H')
    
    # Determine which timestamps are missing
    missing = expected_range.difference(df.index)
    return missing

In [4]:
def preprocess_data(data, price_data=None, on_chain_data=None, set_index=True):
    if price_data and on_chain_data:
        raise ValueError('Invalid Parameter Values. Both price_data and on_chain_data cannot both be True')

    elif (not price_data) and (not on_chain_data):
        raise ValueError('Invalid Parameter Values. Both price_data and on_chain_data cannot both be False')
    
    else:
        # Create the target df and merge on correct dates. Then forward fill the na values
        if price_data:
            df = clean_data(data.copy(), price_data=True, set_index=True) if set_index else clean_data(data.copy(), price_data=True, set_index=False)
            target_df = pd.DataFrame(0, columns=[0], index=pd.date_range(start=df.index.min(), end=df.index.max(), freq='H'))
        
        elif on_chain_data:
            df = clean_data(data.copy(), on_chain_data=True, set_index=True) if set_index else clean_data(data.copy(), on_chain_data=True, set_index=False)
            target_df = pd.DataFrame(0, columns=[0], index=pd.date_range(start=df.index.min(), end=df.index.max(), freq='H'))

        target_df = target_df.join(df, how='left').drop(0, axis=1)
        target_df = target_df.fillna(method='ffill')


        return target_df

BITCOIN CLEANING

In [5]:
btc = pd.read_parquet('BTC_Hourly_On_Chain_Data_sorted.parquet')
btc.head()

,hour,average_height,average_difficulty,average_total_fees,average_total_reward,average_mint_reward,average_transaction_count,average_nonce,average_size,average_stripped_size,average_version,average_weight
3417,2015-01-01 00:00:00.000 UTC,336868.0,4.064096e+10,0.023508,25.023508,25.0,177.733333,1.982945e+09,103153.666667,103153.666667,2.0,4.126147e+05
32629,2015-01-01 01:00:00.000 UTC,336879.0,4.064096e+10,0.043085,25.043085,25.0,309.571429,2.230371e+09,168577.857143,168577.857143,2.0,6.743114e+05
49123,2015-01-01 02:00:00.000 UTC,336886.5,4.064096e+10,0.029944,25.029944,25.0,228.750000,2.312130e+09,155578.500000,155578.500000,2.0,6.223140e+05
45019,2015-01-01 03:00:00.000 UTC,336892.5,4.064096e+10,0.074818,25.074818,25.0,497.000000,1.882359e+09,257693.000000,257693.000000,2.0,1.030772e+06
18681,2015-01-01 04:00:00.000 UTC,336898.0,4.064096e+10,0.032928,25.032928,25.0,246.571429,2.118612e+09,130498.571429,130498.571429,2.0,5.219943e+05


In [6]:
# We must shift the BTC data by 1, since we don't know that information until then

btc['hour'] = pd.to_datetime(btc['hour'])
btc = btc.set_index('hour')
btc = btc.shift(1)
btc = btc.dropna()
display(btc.head())
init_num_missing_hours = len(check_missing_hours(btc))
num_nils_btc = np.where(btc == '<nil>', 1, 0).sum()
print(f'Number of Nils in raw btc on-chain data: {num_nils_btc}')
print(f'Number of missing hours in the raw BTC on-chain data: {init_num_missing_hours}')

,average_height,average_difficulty,average_total_fees,average_total_reward,average_mint_reward,average_transaction_count,average_nonce,average_size,average_stripped_size,average_version,average_weight
hour,,,,,,,,,,,
2015-01-01 01:00:00+00:00,336868.0,4.064096e+10,0.023508,25.023508,25.0,177.733333,1.982945e+09,103153.666667,103153.666667,2.0,4.126147e+05
2015-01-01 02:00:00+00:00,336879.0,4.064096e+10,0.043085,25.043085,25.0,309.571429,2.230371e+09,168577.857143,168577.857143,2.0,6.743114e+05
2015-01-01 03:00:00+00:00,336886.5,4.064096e+10,0.029944,25.029944,25.0,228.750000,2.312130e+09,155578.500000,155578.500000,2.0,6.223140e+05
2015-01-01 04:00:00+00:00,336892.5,4.064096e+10,0.074818,25.074818,25.0,497.000000,1.882359e+09,257693.000000,257693.000000,2.0,1.030772e+06
2015-01-01 05:00:00+00:00,336898.0,4.064096e+10,0.032928,25.032928,25.0,246.571429,2.118612e+09,130498.571429,130498.571429,2.0,5.219943e+05


Number of Nils in raw btc on-chain data: 0
Number of missing hours in the raw BTC on-chain data: 224


In [7]:
btc_chain_processed = preprocess_data(data=btc, on_chain_data=True, set_index=False)
display(btc_chain_processed.head())
num_missing_hours = len(check_missing_hours(btc_chain_processed))
print(f'Number of missing hours in the processed BTC on-chain data: {num_missing_hours}')

,average_height,average_difficulty,average_total_fees,average_total_reward,average_mint_reward,average_transaction_count,average_nonce,average_size,average_stripped_size,average_version,average_weight
2015-01-01 01:00:00+00:00,336868.0,4.064096e+10,0.023508,25.023508,25.0,177.733333,1.982945e+09,103153.666667,103153.666667,2.0,4.126147e+05
2015-01-01 02:00:00+00:00,336879.0,4.064096e+10,0.043085,25.043085,25.0,309.571429,2.230371e+09,168577.857143,168577.857143,2.0,6.743114e+05
2015-01-01 03:00:00+00:00,336886.5,4.064096e+10,0.029944,25.029944,25.0,228.750000,2.312130e+09,155578.500000,155578.500000,2.0,6.223140e+05
2015-01-01 04:00:00+00:00,336892.5,4.064096e+10,0.074818,25.074818,25.0,497.000000,1.882359e+09,257693.000000,257693.000000,2.0,1.030772e+06
2015-01-01 05:00:00+00:00,336898.0,4.064096e+10,0.032928,25.032928,25.0,246.571429,2.118612e+09,130498.571429,130498.571429,2.0,5.219943e+05


Number of missing hours in the processed BTC on-chain data: 0


In [8]:
btc_price = pd.read_csv('btc_hourly.csv').sort_values('time_period_end').set_index('time_period_end')
btc_price.index = pd.to_datetime(btc_price.index)
display(btc_price.head())
num_nans_btc_price = btc.isna().sum().sum()
print(f'Number of NaNs in raw btc price data: {num_nans_btc_price}')
init_num_missing_hours_price = len(check_missing_hours(btc_price))
print(f'Number of missing hours in the raw BTC price data: {init_num_missing_hours_price}')

,time_open,time_close,price_open,price_high,price_low,price_close,volume_traded,trades_count
time_period_end,,,,,,,,
2015-01-14 17:00:00+00:00,2015-01-14T16:07:05.0000000Z,2015-01-14T16:55:16.0000000Z,173.93,197.97,110.20,185.91,1.050000,105
2015-01-14 19:00:00+00:00,2015-01-14T18:50:59.0000000Z,2015-01-14T18:50:59.0000000Z,186.00,186.00,186.00,186.00,0.015362,1
2015-01-14 20:00:00+00:00,2015-01-14T19:05:36.0000000Z,2015-01-14T19:40:43.0000000Z,186.00,188.00,120.00,120.00,0.264638,14
2015-01-15 02:00:00+00:00,2015-01-15T01:35:08.0000000Z,2015-01-15T01:35:20.0000000Z,191.99,192.00,191.99,192.00,1.130000,3
2015-01-15 05:00:00+00:00,2015-01-15T04:09:35.0000000Z,2015-01-15T04:16:31.0000000Z,150.00,150.00,150.00,150.00,0.072200,5


Number of NaNs in raw btc price data: 0
Number of missing hours in the raw BTC price data: 677


In [9]:
btc_price_processed = preprocess_data(data=btc_price, price_data=True, set_index=False)
display(btc_price_processed.head())
num_missing_hours_price = len(check_missing_hours(btc_price_processed))
print(f'Number of missing hours in the processed BTC price data: {num_missing_hours_price}')

,time_open,time_close,price_open,price_high,price_low,price_close,volume_traded,trades_count
2015-01-14 17:00:00+00:00,2015-01-14 16:07:05+00:00,2015-01-14 16:55:16+00:00,173.93,197.97,110.2,185.91,1.050000,105.0
2015-01-14 18:00:00+00:00,2015-01-14 16:07:05+00:00,2015-01-14 16:55:16+00:00,173.93,197.97,110.2,185.91,1.050000,105.0
2015-01-14 19:00:00+00:00,2015-01-14 18:50:59+00:00,2015-01-14 18:50:59+00:00,186.00,186.00,186.0,186.00,0.015362,1.0
2015-01-14 20:00:00+00:00,2015-01-14 19:05:36+00:00,2015-01-14 19:40:43+00:00,186.00,188.00,120.0,120.00,0.264638,14.0
2015-01-14 21:00:00+00:00,2015-01-14 19:05:36+00:00,2015-01-14 19:40:43+00:00,186.00,188.00,120.0,120.00,0.264638,14.0


Number of missing hours in the processed BTC price data: 0


ETHEREUM CLEANING

In [10]:
eth = pd.read_parquet('ETH_Hourly_On_Chain_Data_sorted.parquet').reset_index().drop('index', axis=1).shift(1).iloc[1:]
eth.head()

,hour,average_number,average_gas_limit,average_gas_used,average_difficulty,average_total_difficulty,average_size,average_base_fee_per_gas,average_blob_gas_used,average_excess_blob_gas
1,2015-07-30 15:00:00.000 UTC,465.0,5000.0,0.0,2.159537e+10,9303716647839,700.364909,<nil>,<nil>,<nil>
2,2015-07-30 16:00:00.000 UTC,1631.0,5000.0,0.0,3.852713e+10,44081278443891,672.244476,<nil>,<nil>,<nil>
3,2015-07-30 17:00:00.000 UTC,2862.0,5000.0,0.0,6.932176e+10,107777664952241,644.849858,<nil>,<nil>,<nil>
4,2015-07-30 18:00:00.000 UTC,3824.5,5000.0,0.0,1.082788e+11,191398209046303,642.116628,<nil>,<nil>,<nil>
5,2015-07-30 19:00:00.000 UTC,4588.5,5000.0,0.0,1.511594e+11,289165740480663,625.376133,<nil>,<nil>,<nil>


In [11]:
print(np.where(eth['average_total_difficulty'] == '<nil>', 1, 0).sum())
print(np.where(eth['average_total_difficulty'] == '<nil>'))
print(np.where(eth['average_base_fee_per_gas'] == '<nil>', 1, 0).sum())
print(np.where(eth['average_base_fee_per_gas'] == '<nil>'))
print(np.where(eth['average_blob_gas_used'] == '<nil>', 1, 0).sum())
print(np.where(eth['average_excess_blob_gas'] == '<nil>', 1, 0).sum())

2596
(array([81313, 81314, 81315, ..., 83906, 83907, 83908]),)
52746
(array([    0,     1,     2, ..., 52743, 52744, 52745]),)
75571
75571


In [12]:
# Cutoff ETH at the end because we have NIL values for difficulty
# Get rid of blob metrics because these are only recorded after ethereum moved to proof of stake

eth_cutoff = eth.set_index('hour').loc[:'2022-09-15 07'][['average_number', 'average_gas_limit', 'average_gas_used', 'average_difficulty', 'average_size']]
# Cutoff so we can include average_base_fee_per_gas in our feature set
eth_cutoff.index = pd.to_datetime(eth_cutoff.index)
eth_cutoff.iloc[52740:].head(10)

,average_number,average_gas_limit,average_gas_used,average_difficulty,average_size
hour,,,,,
2021-08-05 06:00:00+00:00,12963407.5,14989228.0,14867784.0,7.710852e+15,76254.156716
2021-08-05 07:00:00+00:00,12963655.0,14988204.0,14647698.0,7.608569e+15,72463.585903
2021-08-05 08:00:00+00:00,12963897.0,14987078.0,14685168.0,7.511196e+15,72078.284047
2021-08-05 09:00:00+00:00,12964170.0,14988947.0,14719515.0,7.590093e+15,69635.636678
2021-08-05 10:00:00+00:00,12964456.5,14987432.0,14872388.0,7.727006e+15,68005.704225
2021-08-05 11:00:00+00:00,12964717.0,14988133.0,14851808.0,7.690777e+15,68376.075949
2021-08-05 12:00:00+00:00,12964974.0,20917874.0,16654806.0,7.696264e+15,70026.660650
2021-08-05 13:00:00+00:00,12965236.0,28436542.0,15070133.0,7.660835e+15,72965.773279
2021-08-05 14:00:00+00:00,12965499.5,27790972.0,14223123.0,7.600428e+15,70116.403571


In [13]:
eth1 = eth_cutoff.loc[:'2021-08-05 11']
eth2 = eth_cutoff.loc['2021-08-05 12':]

In [14]:
num_missing_hours_chain_eth1 = len(check_missing_hours(eth1))
num_missing_hours_chain_eth2 = len(check_missing_hours(eth2))
print(f'Number of missing hours in raw eth1 on-chain data: {num_missing_hours_chain_eth1}')
print(f'Number of missing hours in raw eth2 on-chain data: {num_missing_hours_chain_eth2}')

Number of missing hours in raw eth1 on-chain data: 3
Number of missing hours in raw eth2 on-chain data: 0


In [15]:
eth1_chain_processed = preprocess_data(data=eth1, on_chain_data=True, set_index=False)
eth2_chain_processed = preprocess_data(data=eth2, on_chain_data=True, set_index=False)
num_missing_hours_chain_eth1_processed = len(check_missing_hours(eth1_chain_processed))
print(f'Number of missing hours in raw eth2 on-chain data: {num_missing_hours_chain_eth1_processed}')

Number of missing hours in raw eth2 on-chain data: 0


In [16]:
eth_price = pd.read_csv('eth_hourly.csv').set_index('time_period_end')
eth_price.index = pd.to_datetime(eth_price.index)
num_nans = eth_price.isna().sum().sum()
num_missing_hours_eth_price = len(check_missing_hours(eth_price))
print(f'Number of NaNs in raw eth price data: {num_nans}')
print(f'Number of missing hours in raw eth price data: {num_missing_hours_eth_price}')
eth_price_processed = preprocess_data(data=eth_price, price_data=True, set_index=False)
eth_price_processed.head()

Number of NaNs in raw eth price data: 0
Number of missing hours in raw eth price data: 448


,time_open,time_close,price_open,price_high,price_low,price_close,volume_traded,trades_count
2017-08-17 05:00:00+00:00,2017-08-17 04:00:29.884000+00:00,2017-08-17 04:58:43.046000+00:00,301.13,302.57,298.0,301.61,125.66877,129.0
2017-08-17 06:00:00+00:00,2017-08-17 05:00:33.336000+00:00,2017-08-17 05:59:24.063000+00:00,301.61,303.28,300.0,303.10,377.67246,202.0
2017-08-17 07:00:00+00:00,2017-08-17 06:00:26.088000+00:00,2017-08-17 06:59:53.820000+00:00,302.40,304.44,301.9,302.68,303.86672,182.0
2017-08-17 08:00:00+00:00,2017-08-17 07:00:48.042000+00:00,2017-08-17 07:59:42.019000+00:00,302.68,307.96,302.6,307.96,754.74510,198.0
2017-08-17 09:00:00+00:00,2017-08-17 08:00:04.773000+00:00,2017-08-17 08:59:24.054000+00:00,307.95,309.97,307.0,308.62,150.75029,182.0


SOLANA CLEANING

In [17]:
# Solana On-Chain

raw_sol_chain = pd.read_parquet('query_result_SOL.parquet').set_index('hour').shift(1).iloc[1:]
raw_sol_chain.index = pd.to_datetime(raw_sol_chain.index)
raw_sol_chain

,average_height,average_total_transactions,average_successful_transactions,average_failed_transactions,average_total_vote_transactions,average_total_non_vote_transactions,average_successful_vote_transactions,average_successful_non_vote_transactions,average_failed_vote_transactions,average_failed_non_vote_transactions,average_num_reward_partitions
hour,,,,,,,,,,,
2020-10-03 15:00:00+00:00,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,<nil>
2020-10-03 16:00:00+00:00,<nil>,190.038944,180.470872,9.568072,173.176376,16.862568,165.337625,15.133248,7.838751,1.729321,<nil>
2020-10-03 17:00:00+00:00,<nil>,197.403345,184.241584,13.161762,171.625556,25.777790,161.705060,22.536523,9.920495,3.241266,<nil>
2020-10-03 18:00:00+00:00,<nil>,190.733246,180.610083,10.123164,173.079493,17.653753,164.667941,15.942141,8.411552,1.711612,<nil>
2020-10-03 19:00:00+00:00,<nil>,192.742412,182.575677,10.166735,173.145040,19.597372,164.879100,17.696578,8.265940,1.900794,<nil>
...,...,...,...,...,...,...,...,...,...,...,...
2025-02-23 19:00:00+00:00,3.00883215e+08,1777.877906,1553.820496,224.057410,1313.753719,464.124187,1310.789862,243.030634,2.963857,221.093554,<nil>
2025-02-23 20:00:00+00:00,3.008922995e+08,1758.940510,1546.387618,212.552892,1317.871234,441.069276,1315.451397,230.936222,2.419837,210.133055,<nil>
2025-02-23 21:00:00+00:00,3.009013755e+08,1746.571429,1550.289247,196.282181,1316.694303,429.877125,1314.379664,235.909583,2.314639,193.967543,<nil>


In [18]:
raw_sol_chain = raw_sol_chain[['average_total_transactions', 'average_successful_transactions', 'average_failed_transactions', 'average_total_vote_transactions', 'average_total_non_vote_transactions', 'average_successful_vote_transactions', 'average_successful_non_vote_transactions', 'average_failed_vote_transactions', 'average_failed_non_vote_transactions']]
num_nils_sol_chain = np.where(raw_sol_chain == '<nil>', 1, 0).sum()
print(f'Number of Nils in raw SOL on-chain data: {num_nils_sol_chain}')
init_num_missing_hours_sol_chain = len(check_missing_hours(raw_sol_chain))
print(f'Number of missing hours in the raw SOL on-chain data: {init_num_missing_hours_sol_chain}')

Number of Nils in raw SOL on-chain data: 0
Number of missing hours in the raw SOL on-chain data: 103


In [19]:
sol_chain_processed = preprocess_data(data=raw_sol_chain, on_chain_data=True, set_index=False).iloc[1:]
processed_num_missing_hours_sol_chain = len(check_missing_hours(sol_chain_processed))
print(f'Number of missing hours in the processed SOL on-chain data: {processed_num_missing_hours_sol_chain}')
sol_chain_processed.head()

Number of missing hours in the processed SOL on-chain data: 0


,average_total_transactions,average_successful_transactions,average_failed_transactions,average_total_vote_transactions,average_total_non_vote_transactions,average_successful_vote_transactions,average_successful_non_vote_transactions,average_failed_vote_transactions,average_failed_non_vote_transactions
2020-10-03 16:00:00+00:00,190.038944,180.470872,9.568072,173.176376,16.862568,165.337625,15.133248,7.838751,1.729321
2020-10-03 17:00:00+00:00,197.403345,184.241584,13.161762,171.625556,25.777790,161.705060,22.536523,9.920495,3.241266
2020-10-03 18:00:00+00:00,190.733246,180.610083,10.123164,173.079493,17.653753,164.667941,15.942141,8.411552,1.711612
2020-10-03 19:00:00+00:00,192.742412,182.575677,10.166735,173.145040,19.597372,164.879100,17.696578,8.265940,1.900794
2020-10-03 20:00:00+00:00,190.043383,180.030288,10.013095,173.069072,16.974310,164.780588,15.249700,8.288485,1.724610


In [20]:
# SOL Price Data

raw_sol_price = pd.read_csv('sol_hourly.csv').set_index('time_period_end')
raw_sol_price.index = pd.to_datetime(raw_sol_price.index)
init_num_nans_sol_price = raw_sol_price.isna().sum().sum()
print(f'Number of NaNs in the raw SOL price data: {init_num_nans_sol_price}')
init_num_missing_hours_sol_price = len(check_missing_hours(raw_sol_price))
print(f'Number of missing hours in the raw SOL price data: {init_num_missing_hours_sol_price}')
raw_sol_price.head()

Number of NaNs in the raw SOL price data: 0
Number of missing hours in the raw SOL price data: 388


,time_open,time_close,price_open,price_high,price_low,price_close,volume_traded,trades_count
time_period_end,,,,,,,,
2020-08-11 07:00:00+00:00,2020-08-11T06:03:26.1400000Z,2020-08-11T06:59:15.4480000Z,3.4699,3.4700,2.9054,2.9515,20028.66,266
2020-08-11 08:00:00+00:00,2020-08-11T07:00:03.0470000Z,2020-08-11T07:56:42.0020000Z,2.9515,3.1355,2.8800,2.9224,42069.37,472
2020-08-11 09:00:00+00:00,2020-08-11T08:01:49.5620000Z,2020-08-11T08:58:12.1350000Z,2.9626,3.0000,2.9144,2.9600,24280.76,209
2020-08-11 10:00:00+00:00,2020-08-11T09:00:13.5900000Z,2020-08-11T09:59:37.3670000Z,2.9600,2.9736,2.8500,2.8543,26371.23,230
2020-08-11 11:00:00+00:00,2020-08-11T10:00:04.1750000Z,2020-08-11T10:54:17.0210000Z,2.8566,2.9329,2.8433,2.8976,26685.94,277


In [21]:
sol_price_processed = preprocess_data(data=raw_sol_price, price_data=True, set_index=False)
processed_num_missing_hours_sol_price = len(check_missing_hours(sol_price_processed))
print(f'Number of missing hours in the processed SOL price data: {processed_num_missing_hours_sol_price}')
sol_price_processed.head()

Number of missing hours in the processed SOL price data: 0


,time_open,time_close,price_open,price_high,price_low,price_close,volume_traded,trades_count
2020-08-11 07:00:00+00:00,2020-08-11 06:03:26.140000+00:00,2020-08-11 06:59:15.448000+00:00,3.4699,3.4700,2.9054,2.9515,20028.66,266.0
2020-08-11 08:00:00+00:00,2020-08-11 07:00:03.047000+00:00,2020-08-11 07:56:42.002000+00:00,2.9515,3.1355,2.8800,2.9224,42069.37,472.0
2020-08-11 09:00:00+00:00,2020-08-11 08:01:49.562000+00:00,2020-08-11 08:58:12.135000+00:00,2.9626,3.0000,2.9144,2.9600,24280.76,209.0
2020-08-11 10:00:00+00:00,2020-08-11 09:00:13.590000+00:00,2020-08-11 09:59:37.367000+00:00,2.9600,2.9736,2.8500,2.8543,26371.23,230.0
2020-08-11 11:00:00+00:00,2020-08-11 10:00:04.175000+00:00,2020-08-11 10:54:17.021000+00:00,2.8566,2.9329,2.8433,2.8976,26685.94,277.0


MATIC (POLYGON) CLEANING

In [22]:
# matic on-chain data

raw_matic_chain = pd.read_parquet('query_result_MATIC.parquet').set_index('hour').shift(1).iloc[1:]
raw_matic_chain.index = pd.to_datetime(raw_matic_chain.index)
display(raw_matic_chain)
total_matic_nils = np.where(raw_matic_chain['average_base_fee_per_gas'] == '<nil>', 1, 0).sum()
print(f'Total number of Nils in Raw MATIC on-chain data: {total_matic_nils}')
other_nils_matic = np.where(raw_matic_chain[raw_matic_chain.columns[:-1]] == '<nil>', 1, 0).sum()
print(f'Number of Nils other than average_base_fee_per_gas {other_nils_matic}')
num_missing_hours_chain_matic_raw = len(check_missing_hours(raw_matic_chain))
print(f'Number of missing hours for raw MATIC on-chain data {num_missing_hours_chain_matic_raw}')

,average_number,average_gas_limit,average_gas_used,average_difficulty,average_total_difficulty,average_size,average_base_fee_per_gas
hour,,,,,,,
2020-05-30 17:00:00+00:00,437.5,15477045.0,6451.0,7.000000,3.064000e+03,636.300915,<nil>
2020-05-30 18:00:00+00:00,1747.5,20000000.0,2775.0,7.000000,1.223400e+04,614.375143,<nil>
2020-05-30 19:00:00+00:00,3492.5,20000000.0,63.0,7.000000,2.444900e+04,613.760894,<nil>
2020-05-30 20:00:00+00:00,5237.5,20000000.0,0.0,7.000000,3.666400e+04,613.345361,<nil>
2020-05-30 21:00:00+00:00,6983.5,20000000.0,0.0,7.000000,4.888600e+04,613.345361,<nil>
...,...,...,...,...,...,...,...
2025-02-23 19:00:00+00:00,68284509.5,29982424.0,12926826.0,22.000000,1.303319e+09,55411.429752,435457.3028335301
2025-02-23 20:00:00+00:00,68286203.5,29982424.0,10544614.0,22.000000,1.303357e+09,49007.274498,29.060212514757968
2025-02-23 21:00:00+00:00,68287865.5,29996395.0,10100943.0,21.959509,1.303393e+09,46427.977301,26.47791411042945


Total number of Nils in Raw MATIC on-chain data: 14338
Number of Nils other than average_base_fee_per_gas 0
Number of missing hours for raw MATIC on-chain data 4


In [23]:
# Split MATIC into matic1 and matic2 then process them
matic1_chain = raw_matic_chain.iloc[:14338][raw_matic_chain.columns[:-1]]
matic2_chain = raw_matic_chain.iloc[14338:]

matic1_chain_processed = preprocess_data(data=matic1_chain, on_chain_data=True, set_index=False)
matic2_chain_processed = preprocess_data(data=matic2_chain, on_chain_data=True, set_index=False) #matic2 includes the 'average_base_fee_per_gas' field

num_missing_hours_chain_matic1_processed = len(check_missing_hours(matic1_chain_processed))
num_missing_hours_chain_matic2_processed = len(check_missing_hours(matic2_chain_processed))
print(f'Number of missing hours for processed MATIC1 on-chain data {num_missing_hours_chain_matic1_processed}')
print(f'Number of missing hours for processed MATIC2 on-chain data {num_missing_hours_chain_matic2_processed}')

Number of missing hours for processed MATIC1 on-chain data 0
Number of missing hours for processed MATIC2 on-chain data 0


In [24]:
# matic price data

raw_matic_price = pd.read_csv('matic_hourly.csv').set_index('time_period_end')
raw_matic_price.index = pd.to_datetime(raw_matic_price.index)
init_num_nans_matic_price = raw_matic_price.isna().sum().sum()
print(f'Number of NaNs in the raw MATIC price data: {init_num_nans_matic_price}')
init_num_missing_hours_matic_price = len(check_missing_hours(raw_matic_price))
print(f'Number of missing hours in the raw MATIC price data: {init_num_missing_hours_matic_price}')
matic_price_processed = preprocess_data(data=raw_matic_price, price_data=True, set_index=False)
processed_num_missing_hours_matic_price = len(check_missing_hours(matic_price_processed))
print(f'Number of missing hours in the processed MATIC price data: {processed_num_missing_hours_matic_price}')
matic_price_processed.head()

Number of NaNs in the raw MATIC price data: 0
Number of missing hours in the raw MATIC price data: 440
Number of missing hours in the processed MATIC price data: 0


,time_open,time_close,price_open,price_high,price_low,price_close,volume_traded,trades_count
2019-04-26 16:00:00+00:00,2019-04-26 15:00:00.149000+00:00,2019-04-26 15:59:59.951000+00:00,0.00990,0.01052,0.00526,0.00630,1.429347e+09,17898.0
2019-04-26 17:00:00+00:00,2019-04-26 16:00:00.065000+00:00,2019-04-26 16:59:58.870000+00:00,0.00630,0.00630,0.00560,0.00579,2.190235e+08,3049.0
2019-04-26 18:00:00+00:00,2019-04-26 17:00:04.794000+00:00,2019-04-26 17:59:58.318000+00:00,0.00580,0.00580,0.00513,0.00531,1.739163e+08,1961.0
2019-04-26 19:00:00+00:00,2019-04-26 18:00:01.917000+00:00,2019-04-26 18:59:45.711000+00:00,0.00531,0.00553,0.00497,0.00500,1.195051e+08,1588.0
2019-04-26 20:00:00+00:00,2019-04-26 19:00:00.067000+00:00,2019-04-26 19:59:55.958000+00:00,0.00500,0.00514,0.00481,0.00495,8.282273e+07,1391.0


AVALANCHE CLEANING

In [25]:
#raw_avax_chain = pd.read_parquet('query_result_AVALANCHE.parquet').set_index('hour').shift(1).iloc[1:]
#raw_avax_chain.index = pd.to_datetime(raw_avax_chain.index)
#raw_avax_chain